In [1]:
%load_ext nb_black


def PdfTables(Filename, Date):

    # load the libraries
    import PyPDF2
    import tabula
    import pandas as pd
    import numpy as np
    from numpy import inf

    pd.options.mode.chained_assignment = None  # default='warn'

    # read the pdf file, take the first page only, the second page contains only text, god knows why!!!
    df_org = tabula.read_pdf(Filename, multiple_tables=True, pages="all")
    if len(df_org) == 2:
        df = df_org[0]
        df1 = df_org[1]
    else:
        df = df_org[0]

    # get the original start of the table
    w = np.where(df.iloc[:, 0] == "Party Name")[0].tolist()[0]

    # slice out the table
    df_mod = df.iloc[w + 1 :, :]

    # fix the column names
    df_mod.columns = ["Party Name", "Tenure", "Amount", "Rate", "Date"]

    # turn the Amount column numerical
    df_mod["Amount"] = df_mod["Amount"].str.replace(",", "").fillna(0).astype("float64")

    # split and fix the Deal Date and Maturity Date column (which previously came as a single column)
    date = df_mod["Date"].str.split(" ", n=1, expand=True)
    df_mod["Deal Date"] = date[0]
    df_mod["Maturity Date"] = date[1]
    df_mod.drop("Date", axis=1, inplace=True)
    df_mod = df_mod[pd.notnull(df_mod["Deal Date"])]
    df_mod["Deal Date"] = pd.to_datetime(df_mod["Deal Date"], dayfirst=True)
    df_mod["Maturity Date"] = pd.to_datetime(df_mod["Maturity Date"], dayfirst=True)

    # For Call Money Lending the Maturity day is the next day of the Deal Date
    df_mod["Maturity Date"] = df_mod["Maturity Date"].fillna(
        df_mod["Deal Date"] + pd.Timedelta(days=1)
    )
    df_mod.drop("Tenure", axis=1, inplace=True)

    # Get the remaining day for maturity
    df_mod["Maturity Bucket"] = df_mod["Maturity Date"] - pd.to_datetime(
        Date, format="%Y-%m-%d"
    )
    df_mod["Maturity_Bucket"] = df_mod["Maturity Bucket"].dt.days

    # Get the summary output
    inv_sum = (
        df_mod.groupby(["Party Name", "Maturity Bucket"])["Amount"]
        .agg(["sum"])
        .rename(columns={"sum": "Total Balance"})
    )
    inv_sum["Total Balance"] = inv_sum["Total Balance"] / 10000000
    output = inv_sum.round({"Total Balance": 2})

    # Get the Maturity Bucket segregated by yield type
    # Maturity Bucket set
    mat_lev = [-inf, 7, 30, 60, 90, 120, 150, 180, inf]
    mat_lab = [
        "<1 Week",
        "1 Week-1 Month",
        "1 Month-2 Month",
        "2 Month-3 Month",
        "3 Month-4 Month",
        "4 Month-5 Month",
        "5 Month-6 Month",
        ">6 Month",
    ]
    df_mod["Mat_Buc"] = pd.cut(df_mod["Maturity_Bucket"], bins=mat_lev, labels=mat_lab)

    # Get the Yield Bucket
    df_mod["Rate"] = df_mod["Rate"].astype("float64")
    rate_lev = [0, 5, 9, inf]
    rate_lab = [
        "Low Yield (0.00%-5.00%)",
        "Mid Yield (5.01%-9.00%)",
        "High Yield (>9.00%)",
    ]
    df_mod["Yield_Buc"] = pd.cut(df_mod["Rate"], bins=rate_lev, labels=rate_lab)
    Output_Summary = (
        df_mod.groupby(["Mat_Buc", "Yield_Buc"])["Amount"]
        .agg(["sum"])
        .rename(columns={"sum": "Total_Balance"})
    )

    # Get the required Summary Table
    Output_Summary["Total_Balance"] = Output_Summary["Total_Balance"] / 10000000
    Output_Summary["Total_Balance"] = Output_Summary.round({"Total_Balance": 2})
    return Output_Summary

<IPython.core.display.Javascript object>

In [2]:
PdfTables("F:\GoogleDrive\python\Pdf\Position.pdf", "2019-10-31")

Total_Balance
Mat_Buc         Yield_Buc                             
<1 Week         Low Yield (0.00%-5.00%)         682.69
1 Week-1 Month  Mid Yield (5.01%-9.00%)         100.00
                High Yield (>9.00%)              75.00
2 Month-3 Month Mid Yield (5.01%-9.00%)          50.00
                High Yield (>9.00%)             325.00

<IPython.core.display.Javascript object>